In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy
import geopandas as gpd
import xarray as xr
import sparse
from itertools import product, combinations
from pathlib import Path
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib
from collections import defaultdict
from ortools.sat.python import cp_model

pd.options.display.max_rows = 500
#pd.options.display.max_columns = 4000

import sys
sys.path.append('../src/')

from extended_survey import process_people_df, process_places_df, categorize_p, categorize_v
from census import process_census, merge_loc_agebs
from constraints import get_ind_const, get_viv_const
from misc import mun_dict, met_zone
from taz import load_marco_geo, merge_mg_taz, plot_taz_mg, plot_taz_mg_unass, plot_taz_empty_mg, plot_taz_codes, plot_chull

survey_dir = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/')
personas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Personas19.CSV')
viviendas_path = Path('../data/cuestionario_ampliado/Censo2020_CA_nl_csv/Viviendas19.CSV')
census_iter_path = Path('../data/census_loc/ITER_19CSV20.csv')
census_resageburb_path = Path('../data/census_ageb_manz/RESAGEBURB_19CSV20.csv')
output_path = Path('../output/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
(
    df_mun, df_loc,
    df_agebs
) = process_census(census_iter_path, census_resageburb_path)

df_loc_agebs = merge_loc_agebs(df_mun, df_loc, df_agebs, impute=True)

In [5]:
marco_geo_path = Path('../data/19_nuevoleon.gpkg')
mg = load_marco_geo(marco_geo_path, df_mun, df_loc_agebs)

taz = gpd.read_file('/Users/gperaza/Downloads/TAZ/Zonas.shp').to_crs(mg.crs).drop(columns='ESTADO')
mtaz = gpd.read_file('/Users/gperaza/Downloads/TAZ/Macrozonas.shp').to_crs(mg.crs)

In [6]:
from matplotlib.backends.backend_pdf import PdfPages
import warnings

In [9]:
overlay_dict = {}
taz_dict = {}
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    with PdfPages('multipage_pdf.pdf') as pdf:
        for mun in met_zone:
            taz_mun = taz[taz.MUNICIPIO == mun].copy()
            taz_dict[mun] = taz_mun.set_index('ZONA').sort_index()
            overlay = merge_mg_taz(mun, taz, mg)
            overlay_dict[mun] = overlay
            
            fig, ax = plt.subplots(2, 2, figsize=(20, 20))

            plot_taz_mg(overlay, taz_mun, title=mun, ax=ax[0, 0])
            plot_taz_mg_unass(overlay, taz_mun, title='Unassigned AGEBS', ax=ax[0, 1])
            plot_taz_empty_mg(overlay, taz_mun, title='Empty TAZs', ax=ax[1, 0])
            plot_chull(taz_mun, title='Multipol TAZs', ax=ax[1, 1])

            pdf.savefig()

            plt.close()

In [10]:
for mun, ovr in overlay_dict.items():
    taz_dict[mun] = taz_dict[mun].join(overlay_dict[mun].groupby('ZONA')[df_agebs.columns].sum().astype(int), how='outer')
    assert taz_dict[mun].index.is_unique
assert(pd.concat(taz_dict.values()).drop(-10).shape[0] == taz.dropna().shape[0]) 

In [11]:
for mun, gdf in taz_dict.items():
    gdf.to_file("taz_census.gpkg", layer=mun, driver="GPKG") 